In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict
from math import sqrt
import matplotlib.pyplot as plt

df = pd.read_csv("C:\\Users\\NMPL-SKKWON\\Desktop\\국민건강보험공단_건강검진정보_20221231.CSV", encoding='cp949')
print(df.head())


C:\Users\NMPL-SKKWON\AppData\Local\Temp\ipykernel_16908\1611141860.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


   기준년도  가입자일련번호  시도코드  성별  연령대코드(5세단위)  신장(5cm단위)  체중(5kg단위)  허리둘레  시력(좌)  \
0  2022   702763    41   2           14        160         75  89.0    1.0   
1  2022   765435    41   2            9        150         45  68.0    1.0   
2  2022   343621    41   2           13        150         60  78.8    1.0   
3  2022  3393534    41   2            6        160         50  69.0    1.0   
4  2022  1716951    41   2            7        160         55  75.0    1.0   

   시력(우)  ...  요단백  혈청크레아티닌  혈청지오티(AST)  혈청지피티(ALT)  감마지티피  흡연상태  음주여부  \
0    1.0  ...  1.0      0.8        25.0        20.0   15.0   1.0   0.0   
1    1.0  ...  1.0      0.8        19.0        10.0    7.0   1.0   1.0   
2    1.0  ...  1.0      0.8        17.0        15.0   15.0   1.0   1.0   
3    1.0  ...  1.0      0.8        25.0        16.0   13.0   1.0   1.0   
4    1.0  ...  1.0      0.8        22.0        22.0   15.0   1.0   1.0   

   구강검진수검여부  치아우식증유무  치석  
0         0      NaN NaN  
1         0      NaN NaN  
2    

In [2]:
df = df.drop(columns=['총콜레스테롤', '트리글리세라이드', 'HDL콜레스테롤', 'LDL콜레스테롤', '치아우식증유무', '치석', '구강검진수검여부', '시력(좌)', '시력(우)', '청력(좌)', '청력(우)', '시도코드', '기준년도'])

In [3]:
# 수축기혈압과 이완기혈압 열에 NaN 값이 존재하는 행 제거
df_filtered = df.dropna(subset=['수축기혈압', '이완기혈압'])
print("\n수축기혈압 또는 이완기혈압에 NaN 값이 있는 행 제거 후 DataFrame:")
print(df_filtered)



수축기혈압 또는 이완기혈압에 NaN 값이 있는 행 제거 후 DataFrame:
        가입자일련번호  성별  연령대코드(5세단위)  신장(5cm단위)  체중(5kg단위)   허리둘레  수축기혈압  이완기혈압  \
0        702763   2           14        160         75   89.0  127.0   73.0   
1        765435   2            9        150         45   68.0  127.0   75.0   
2        343621   2           13        150         60   78.8  135.0   81.0   
3       3393534   2            6        160         50   69.0  109.0   70.0   
4       1716951   2            7        160         55   75.0  110.0   67.0   
...         ...  ..          ...        ...        ...    ...    ...    ...   
999995  2134210   1            6        170         90   98.8  128.0   84.0   
999996  1686113   2           15        150         85  116.0  136.0   81.0   
999997   902951   1           12        165         80   98.2  119.0   83.0   
999998  3721315   1            6        175         65   75.4  109.0   60.0   
999999  4901217   2           14        150         40   74.1  137.0   87.0   

      

In [4]:
# 전체 행에 NaN 값이 두 개 이상 존재하는 행 제거
df_filtered = df_filtered.dropna(thresh=df_filtered.shape[1] - 1)
print("\n전체 행에 NaN 값이 두 개 이상 존재하는 행 제거 후 DataFrame:")
print(df_filtered)



전체 행에 NaN 값이 두 개 이상 존재하는 행 제거 후 DataFrame:
        가입자일련번호  성별  연령대코드(5세단위)  신장(5cm단위)  체중(5kg단위)   허리둘레  수축기혈압  이완기혈압  \
0        702763   2           14        160         75   89.0  127.0   73.0   
1        765435   2            9        150         45   68.0  127.0   75.0   
2        343621   2           13        150         60   78.8  135.0   81.0   
3       3393534   2            6        160         50   69.0  109.0   70.0   
4       1716951   2            7        160         55   75.0  110.0   67.0   
...         ...  ..          ...        ...        ...    ...    ...    ...   
999995  2134210   1            6        170         90   98.8  128.0   84.0   
999996  1686113   2           15        150         85  116.0  136.0   81.0   
999997   902951   1           12        165         80   98.2  119.0   83.0   
999998  3721315   1            6        175         65   75.4  109.0   60.0   
999999  4901217   2           14        150         40   74.1  137.0   87.0   

       

In [5]:
df = df_filtered

In [6]:
from sklearn.impute import SimpleImputer, KNNImputer


df['고혈압유무'] = df.apply(lambda row: 1 if row['수축기혈압'] >= 140 and row['이완기혈압'] >= 90 else 0, axis=1)
# 데이터 정규화

In [7]:
df = df[df['연령대코드(5세단위)'] >= 4]

In [8]:
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
import pandas as pd

file_paths = ["C:\\Users\\NMPL-SKKWON\\Desktop\\국민건강보험공단_의약품처방정보_20221231\\3.국민건강보험공단_의약품처방정보_2022_2(수정).CSV"]

# 모든 파일을 읽어서 하나의 DataFrame으로 합치기
dfs = [pd.read_csv(file, encoding='cp949') for file in file_paths]
df2 = pd.concat(dfs, ignore_index=True)
antihypertensive_atc_codes = [
    'C02', 'C03', 'C07', 'C08', 'C09'
    # 필요한 경우에 따라 더 추가할 수 있습니다.
]
# file2에서 고혈압 치료제에 해당하는 ATC 코드를 가진 행 필터링
antihypertensive_df2 = df2[df2['약품일반성분명코드(ATC코드)'].str.startswith(tuple(antihypertensive_atc_codes))]
# file1에서 고혈압 치료제에 해당하는 ATC 코드를 가진 행들의 일련번호 가져오기
antihypertensive_serial_numbers = antihypertensive_df2['가입자일련번호'].unique()
# file1에서 고혈압 치료제에 해당하는 ATC 코드를 가진 행 제거
df = df[~df['가입자일련번호'].isin(antihypertensive_serial_numbers)]

# 결과 확인
print(df.head())
# 일련번호를 기준으로 두 데이터프레임을 병합 (inner join)
merged_df = pd.merge(df, df2[['가입자일련번호', '약품일반성분명코드(ATC코드)']], on='가입자일련번호', how='inner')
merged_df = merged_df.drop_duplicates(subset=['가입자일련번호'])
# 결과 확인
print(merged_df.head())


In [ ]:
merged_df

In [ ]:
df = merged_df.reset_index(drop=True)

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.svm import SVC
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [19]:

# X와 y 분리
x_columns = ['신장(5cm단위)', '체중(5kg단위)', '혈청지피티(ALT)', '혈청지오티(AST)', '혈청크레아티닌', '연령대코드(5세단위)', 
             '허리둘레', '식전혈당(공복혈당)', '혈색소', '감마지티피']
X = df[x_columns]
y = df['고혈압유무']

X = pd.concat([X, df[['성별', '흡연상태', '음주여부', '요단백']]], axis=1)

imputer = SimpleImputer(strategy='median')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)


data = pd.concat([X, df[['고혈압유무']]], axis=1)

X['성별'] = np.round(X['성별'])
X['흡연상태'] = np.round(X['흡연상태'])
X['음주여부'] = np.round(X['음주여부'])
X['요단백'] = np.round(X['요단백'])
X['연령대코드(5세단위)'] = np.round(X['연령대코드(5세단위)'])
# 평균값으로 보간 






In [20]:
X

,신장(5cm단위),체중(5kg단위),혈청지피티(ALT),혈청지오티(AST),혈청크레아티닌,연령대코드(5세단위),허리둘레,식전혈당(공복혈당),혈색소,감마지티피,성별,흡연상태,음주여부,요단백
0,160.0,75.0,20.0,25.0,0.8,14.0,89.0,84.0,14.8,15.0,2.0,1.0,0.0,1.0
1,150.0,45.0,10.0,19.0,0.8,9.0,68.0,85.0,13.1,7.0,2.0,1.0,1.0,1.0
2,150.0,60.0,15.0,17.0,0.8,13.0,78.8,105.0,13.0,15.0,2.0,1.0,1.0,1.0
3,160.0,50.0,16.0,25.0,0.8,6.0,69.0,81.0,13.4,13.0,2.0,1.0,1.0,1.0
4,160.0,55.0,22.0,22.0,0.8,7.0,75.0,79.0,14.7,15.0,2.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994332,170.0,90.0,251.0,95.0,0.9,6.0,98.8,88.0,15.8,77.0,1.0,1.0,1.0,1.0
994333,150.0,85.0,49.0,59.0,1.4,15.0,116.0,134.0,12.4,25.0,2.0,1.0,0.0,1.0
994334,165.0,80.0,82.0,48.0,1.0,12.0,98.2,181.0,15.7,60.0,1.0,2.0,1.0,1.0
994335,175.0,65.0,72.0,245.0,1.1,6.0,75.4,91.0,13.7,17.0,1.0,2.0,1.0,1.0


In [26]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# 2. 클래스 불균형 확인
target_column = '고혈압유무'  # 타겟 컬럼의 이름을 지정해주세요.
print(f'원본 데이터 클래스 분포: {Counter(data[target_column])}')

# 3. Random Over-Sampling 적용
ros = RandomOverSampler(random_state=42)
X = data.drop(columns=[target_column])
y = data[target_column]
X_resampled, y_resampled = ros.fit_resample(X, y)

# 4. 결과 확인
print(f'오버 샘플링 후 클래스 분포: {Counter(y_resampled)}')

# Resampled 데이터프레임 생성
resampled_data = pd.DataFrame(X_resampled, columns=X.columns)
resampled_data[target_column] = y_resampled

원본 데이터 클래스 분포: Counter({0: 938808, 1: 55529})
오버 샘플링 후 클래스 분포: Counter({0: 938808, 1: 938808})


In [ ]:
X_resampled

In [30]:
scaler = StandardScaler()
X_resampled = scaler.fit_transform(X_resampled)
# 3. 5-Fold 교차 검증 설정
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4. RandomForest 분류기 설정
clf = RandomForestClassifier(random_state=42)

# 5. 하드 보팅을 위한 예측 초기화
y_preds = np.zeros(len(y_resampled))

# 6. 5-Fold 교차 검증 수행
for fold, (train_index, test_index) in enumerate(kf.split(X_resampled, y_resampled)):
    print(f'Fold {fold + 1}')
    X_train, X_test = X_resampled[train_index], X_resampled[test_index]
    y_train, y_test = y_resampled[train_index], y_resampled[test_index]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_preds[test_index] = y_pred
    print(f'Fold {fold + 1} predictions: {y_pred[:10]}')  # 첫 10개 예측 출력

# 7. 최종 성능 평가
accuracy = accuracy_score(y_resampled, y_preds)
f1 = f1_score(y_resampled, y_preds)
cm = confusion_matrix(y_resampled, y_preds)

print("Confusion Matrix:")
print(cm)
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Fold 1
Fold 1 predictions: [0 0 0 0 0 1 0 0 0 0]
Fold 2
Fold 2 predictions: [0 0 0 0 0 0 0 0 0 0]
Fold 3
Fold 3 predictions: [0 0 0 0 0 1 0 0 0 0]
Fold 4
Fold 4 predictions: [0 0 0 0 0 0 0 0 0 0]
Fold 5
Fold 5 predictions: [0 0 0 0 0 0 0 0 0 0]
Confusion Matrix:
[[937979    829]
 [     0 938808]]
Accuracy: 0.9996
F1 Score: 0.9996


In [18]:
y_soft_vote

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
def golden_test(data, feature, column, save, y):
    classifiers = {
        "Random Forest": RandomForestClassifier(n_estimators=5, random_state=42)
    }
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    results = {}
    if(len(save) > 0):
        feature = pd.concat([feature, save], axis=1) 
    feature = pd.concat([feature, data[column]], axis=1) 
    print(feature)
    scaler = StandardScaler()
    feature = scaler.fit_transform(feature)
    for name, clf in classifiers.items():
        y_pred = np.zeros(len(y))
        for train_index, test_index in kf.split(feature, y):
            X_train, X_test = feature[train_index], feature[test_index]
            y_train, y_test = y[train_index], y[test_index]
            clf.fit(X_train, y_train)
            y_pred[test_index] = clf.predict(X_test)
        
        cm = confusion_matrix(y, y_pred)
        accuracy = accuracy_score(y, y_pred)
        results[name] = (cm, accuracy)
        print(f"{name} Confusion Matrix:")
        print(cm)
        print(f"Accuracy: {accuracy:.4f}\n")
    return accuracy
column = []
column = X.columns
l = len(column)
t_column = pd.DataFrame()
b_column = pd.DataFrame()
b_rmse = -1
temp = 0
b_feat = ''
for i in range(0, l):
    for r in range (0, (l - i)):
        temp = golden_test(X_resampled, t_column, column[r], b_column, y_resampled)
        if(temp > b_rmse):
            b_rmse = temp
            b_feat = column[r]
        t_column = pd.DataFrame()
    b_rmse = -1
    column  = list(filter(lambda col: col != b_feat, column))
    print(column)
    b_column = pd.concat([b_column, X_resampled[b_feat]], axis=1)  
print(b_column)


In [ ]:
b_column